In [6]:
# write a script to evaluate Qwen 2.5 Math 1.5B zero-shot performance on gsm8k
# Load the gsm8k examples from file.
from datasets import load_dataset
import re

main_ds = load_dataset(
    "parquet",
    data_files = {
        "train": "/home/zhangwj/gsm8k/main/train-00000-of-00001.parquet",
        "test": "/home/zhangwj/gsm8k/main/test-00000-of-00001.parquet",
    }
)

socratic_ds = load_dataset(
    "parquet",
    data_files={
        "train": "/home/zhangwj/gsm8k/socratic/train-00000-of-00001.parquet",
        "test": "/home/zhangwj/gsm8k/socratic/test-00000-of-00001.parquet",
    }
)


In [2]:
import torch
y = torch.tensor([1,2,3,4,5,6,7,8,9,10])
x = y.reshape(2, 5)
print(x)
z = x.reshape(10,)
print(z)

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])


In [3]:
import torch

# Create two tensors
x = torch.tensor([1, 15, 3, 20])
y = torch.tensor([5, 10, 8, 12])

# Find the element-wise minimum
z = torch.min(x, y)

print(z)
# Output: tensor([1, 10, 3, 12])

tensor([ 1, 10,  3, 12])


In [ ]:
print(main_ds['train'])

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


In [7]:
from datasets import load_dataset
file_path = "/home/zhangwj/assignment5/data/gsm8k/test.jsonl"
dataset = load_dataset('json', data_files=file_path)
print(dataset["train"][4])

{'question': "Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens?", 'answer': 'If each chicken eats 3 cups of feed per day, then for 20 chickens they would need 3*20=<<3*20=60>>60 cups of feed per day.\nIf she feeds the flock 15 cups of feed in the morning, and 25 cups in the afternoon, then the final meal would require 60-15-25=<<60-15-25=20>>20 cups of chicken feed.\n#### 20'}


In [1]:
print([1, 2] + [3, 4])

[1, 2, 3, 4]


In [8]:
main_ds = main_ds["test"]

def format_r1_zero(example):
    # Separate reasoning from final answe
    answer_text = example["answer"].strip()
    match = re.search(r"####\s*(.*)", answer_text)
    if match:
        final_answer = match.group(1).strip()
        reasoning = answer_text[:match.start()].strip()
    else:
        reasoning = answer_text
        final_answer = ""
    
    # Build the r1_zero_style prompt
    prompt = (
        "A conversation between User and Assistant. The User asks a question, "
        "and the Assistant solves it. The Assistant first thinks about the reasoning "
        "process in the mind and then provides the User with the answer. "
        "The reasoning process is enclosed within <think> </think> and answer is enclosed "
        "within <answer> </answer> tags, respectively, "
        "i.e., <think> reasoning process here </think> <answer> answer here </answer>.\n\n"
        f"User: {example['question']}\n"
        f"Assistant: <think> {reasoning} </think> <answer> {final_answer} </answer>"
    )
    return {"prompt": prompt}

# Apply to the dataset
formatted_ds = main_ds.map(format_r1_zero)

print(formatted_ds[0]["prompt"])


Map: 100%|██████████| 1319/1319 [00:00<00:00, 36287.05 examples/s]

A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.

User: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Assistant: <think> Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market. </think> <answer> 18 </answer>


In [ ]:
from vllm import LLM, SamplingParams

def main():
    # Sample prompts.
    prompts = [
        "Hello, my name is",
        "The president of the United States is",
        "The capital of France is",
        "The future of AI is",
    ]

    # Create a sampling params object, stopping generation on new line.
    sampling_params = SamplingParams(
        temperature=1.0, top_p=1.0, max_tokens=1024, stop=["\n"]
    )

    # Create an LLM
    llm = LLM(model="/home/zhangwj/Qwen2.5-3B-Instruct")
    # llm = LLM(model="./Qwen2.5-Math-1.5B")

    # Generate texts from the prompts. The output is a list of RequestOutput objects
    # that contain the prompt, generated text, and other information.

    outputs = llm.generate(prompts, sampling_params)


    # Print the outputs.
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

: 